# 基于PaddleSeg的遥感地块分割

## 一、PaddleSeg安装及环境配置


In [1]:
!pip install paddleseg

## 二、数据处理

### 解压数据集

In [2]:
!unzip -q data/data77571/train_and_label.zip
!unzip -q data/data77571/img_test.zip

### 数据处理
项目使用PaddleX进行数据处理和训练。

对用于分割任务的数据进行操作。可以利用paddlex.seg.transforms中的Compose类将图像预处理/增强操作进行组合。

- RandomHorizontalFlip 以一定的概率对图像进行水平翻转。
- Resize 调整图像大小。
- RandomPaddingCrop 对图像和标注图进行随机裁剪，当所需要的裁剪尺寸大于原图时，则进行padding操作，模型训练时的数据增强操作。
- RandomBlur 以一定的概率对图像进行高斯模糊。
- RandomRotate 对图像进行随机旋转，模型训练时的数据增强操作。目前支持多通道的RGB图像，例如支持多张RGB图像沿通道轴做concatenate后的图像数据，不支持通道数量不是3的倍数的图像数据。
- RandomDistort 以一定的概率对图像进行随机像素内容变换，模型训练时的数据增强操作。目前支持多通道的RGB图像，例如支持多张RGB图像沿通道轴做concatenate后的图像数据，不支持通道数量不是3的倍数的图像数据。
- Normalize 对图像进行标准化。